In [ ]:
import datetime as dt
import dlib
import cv2
import numpy as np

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(PREDICTOR_PATH)
detector = dlib.get_frontal_face_detector()

In [ ]:
distraction_start_time = None
distraction_time_elapsed = None
distraction_curr_time = None
distraction_warning = "Person is not distracted."
distraction_color = (0, 255, 0)

yawn_start_time = None
yawn_time_elapsed = None
yawn_curr_time = None
yawn_warning = "Person is not yawning."
yawn_color = (0, 255, 0)

doze_start_time = None
doze_time_elapsed = None
doze_curr_time = None
doze_warning = "Person is awake."
doze_color = (0, 255, 0)

now_time = None
video_capture = cv2.VideoCapture(0)
        
while True:
    ret, image = video_capture.read()
    dim = image.shape
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ratio = 255 / np.amax(gray)
    gray = (gray * ratio).astype(np.uint8)
    rects, scores, idx = detector.run(gray, 1, 0.25)
    now_time = dt.datetime.now()

    if len(rects) == 0:
        if distraction_start_time == None:
            distraction_start_time = now_time
            distraction_curr_time = now_time
            distraction_time_elapsed = dt.timedelta()
        else:
            distraction_time_diff = now_time - distraction_curr_time
            distraction_curr_time = now_time
            distraction_time_elapsed += distraction_time_diff
            if distraction_time_diff < dt.timedelta(seconds = 5):
                if distraction_time_elapsed > dt.timedelta(seconds = 10):
                    distraction_warning = "Level 2 Distraction detected."
                    distraction_color = (0, 0, 255)
                elif distraction_time_elapsed > dt.timedelta(seconds = 5):
                    distraction_warning = "Level 1 Distraction detected."
                    distraction_color = (0, 255, 255)
    elif distraction_start_time != None and (now_time - distraction_curr_time) >= dt.timedelta(seconds = 2):
        distraction_start_time = None
        distraction_warning = "Person is not distracted."
        distraction_color = (0, 255, 0)

    factor1 = 0.1
    factor2 = 0.35

    if len(rects) > 0:
        max_rect = None
        max_rect_size = 0

        for i, rect in enumerate(rects):

            landmarks = [[p.x, p.y] for p in predictor(image, rect).parts()]

            x = rect.left()
            y = rect.top()
            w = rect.right() - x
            h = rect.bottom() - y
            area = w * h
            if max_rect_size < area:
                max_rect_size = area
                max_rect = (landmarks, x, y, w, h)

        landmarks, x, y, w, h = max_rect
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 1)

        le1x, le1y = landmarks[36]
        re1x, re1y = landmarks[39]
        u1e1x, u1e1y = landmarks[37]
        u2e1x, u2e1y = landmarks[38]
        d1e1x, d1e1y = landmarks[40]
        d2e1x, d2e1y = landmarks[41]
        ue1x, ue1y, de1x, de1y = (u1e1x + u2e1x) // 2, (u1e1y + u2e1y) // 2, (d1e1x + d2e1x) // 2, (d1e1y + d2e1y) // 2

        le2x, le2y = landmarks[42]
        re2x, re2y = landmarks[45]
        u1e2x, u1e2y = landmarks[43]
        u2e2x, u2e2y = landmarks[44]
        d1e2x, d1e2y = landmarks[46]
        d2e2x, d2e2y = landmarks[47]
        ue2x, ue2y, de2x, de2y = (u1e2x + u2e2x) // 2, (u1e2y + u2e2y) // 2, (d1e2x + d2e2x) // 2, (d1e2y + d2e2y) // 2

        llx, lly = landmarks[60]
        rlx, rly = landmarks[64]
        ulx, uly = landmarks[62]
        dlx, dly = landmarks[66]

        slope = float(rly - lly) / float(rlx - llx)
        intercept = slope * (-1) * rlx + rly
        dist1 = (slope * ulx - uly + intercept) / ((1 + slope ** 2) ** 0.5)
        dist2 = (slope * dlx - dly + intercept) / ((1 + slope ** 2) ** 0.5)
        left_to_right1 = ((llx - rlx) ** 2 + (lly - rly) ** 2) ** 0.5

        factor1 = (dist1 - dist2) / left_to_right1

        if factor1 > 0.15:
            if yawn_start_time == None:
                yawn_start_time = now_time
                yawn_curr_time = now_time
                yawn_time_elapsed = dt.timedelta()
            else:
                yawn_time_diff = now_time - yawn_curr_time
                yawn_curr_time = now_time
                yawn_time_elapsed += yawn_time_diff
                if yawn_time_diff < dt.timedelta(seconds = 6):
                    if yawn_time_elapsed > dt.timedelta(seconds = 12):
                        yawn_warning = "Level 2 Yawn detected."
                        yawn_color = (0, 0, 255)
                    elif yawn_time_elapsed > dt.timedelta(seconds = 6):
                        yawn_warning = "Level 1 Yawn detected."
                        yawn_color = (0, 255, 255)
        elif yawn_start_time != None and (now_time - yawn_curr_time) >= dt.timedelta(seconds = 2):
            yawn_start_time = None
            yawn_warning = "Person is not yawning."
            yawn_color = (0, 255, 0)

        slope = float(re1y - le1y) / float(re1x - le1x)
        intercept = slope * (-1) * re1x + re1y
        dist3 = (slope * ue1x - ue1y + intercept) / ((1 + slope ** 2) ** 0.5)
        dist4 = (slope * de1x - de1y + intercept) / ((1 + slope ** 2) ** 0.5)
        left_to_right2 = ((le1x - re1x) ** 2 + (le1y - re1y) ** 2) ** 0.5

        slope = float(re2y - le2y) / float(re2x - le2x)
        intercept = slope * (-1) * re2x + re2y
        dist5 = (slope * ue2x - ue2y + intercept) / ((1 + slope ** 2) ** 0.5)
        dist6 = (slope * de2x - de2y + intercept) / ((1 + slope ** 2) ** 0.5)
        left_to_right3 = ((le2x - re2x) ** 2 + (le2y - re2y) ** 2) ** 0.5

        factor2 = ((dist3 - dist4) / left_to_right2 + (dist5 - dist6) / left_to_right3) / 2
        
        if factor2 < 0.26:
            if doze_start_time == None:
                doze_start_time = now_time
                doze_curr_time = now_time
                doze_time_elapsed = dt.timedelta()
            else:
                doze_time_diff = now_time - doze_curr_time
                doze_curr_time = now_time
                doze_time_elapsed += doze_time_diff
                if doze_time_diff < dt.timedelta(seconds = 5):
                    if doze_time_elapsed > dt.timedelta(seconds = 10):
                        doze_warning = "Level 2 Dozing off detected."
                        doze_color = (0, 0, 255)
                    elif doze_time_elapsed > dt.timedelta(seconds = 5):
                        doze_warning = "Level 1 Dozing off detected."
                        doze_color = (0, 255, 255)
        elif doze_start_time != None and (now_time - doze_curr_time) >= dt.timedelta(seconds = 2):
            doze_start_time = None
            doze_warning = "Person is not Dozing off."
            doze_color = (0, 255, 0)

    cv2.putText(image, distraction_warning, (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, distraction_color)
    cv2.putText(image, yawn_warning, (5, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, yawn_color)
    cv2.putText(image, doze_warning, (5, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, doze_color)

    cv2.imshow("Video", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()